In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from numpy import loadtxt
from keras.models import load_model

In [2]:
data = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/only_up.csv', low_memory=False, header=None)

In [3]:
model=load_model('/home/jordan/Documents/Data-Science/Medium/model.h5')

2022-03-05 22:28:57.999528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 22:28:58.006851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 22:28:58.007036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-05 22:28:58.007508: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               768       
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 17,409
Trainable params: 17,409
Non-trainable params: 0
_________________________________________________________________


In [5]:
def snip(data):
    time= data.iloc[:,0]
    time=time.map(lambda x: str(x[6:]))
    time=time.map(lambda x: float(x))

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    #status=status.rolling(20).mean()
    #status=status.map(np.floor)

    train_init=pd.concat([time, interface, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time', 'Interface','Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    train_label=train.filter(['Status'])
    train=train.drop(['Status', 'Inband', 'Outband'], axis=1)
    train=train.reset_index(drop=True)

    return train, train_label

In [6]:
def preproccess(data):
    train,train_label=snip(data)
    train=train.iloc[:-700, :]
    train=train.to_numpy()
    train_label=train_label.iloc[700:]
    return train, train_label

In [7]:
train, train_label = preproccess(data)

In [8]:
score = model.evaluate(train, train_label, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

2022-03-05 22:28:58.898033: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-03-05 22:28:58.898060: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-03-05 22:28:58.898076: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:442 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'sequential/dense/MatMul' defined at (most recent call last):
    File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jordan/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/jordan/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/jordan/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/jordan/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.9/asyncio/base_events.py", line 596, in run_forever
      self._run_once()
    File "/usr/lib